## Problem K-Kineskih poštara

Problem k-kineskih poštara *(Minimum k-Chinese Postman Problem, k-CPP)* je poznati problem kombinatorne optimizacije, izveden je iz problema kineskog poštara *(The Chinese Postman Problem - CPP)* koji je definisao kineski matematičar Guan još 1960. godine.

Problem k-Kineskih poštara adresira potrebu iz stvarnog sveta da poštanska ustanova želi da na jedan teren pošalje više od jednog poštara. Stoga se i cilj menja, potrebno je minimizirati ukupnu dužinu puta koju prelaze svi poštari zajedno.

### Ulazni parametri

Problem k-Kineskih poštara adresira potrebu iz stvarnog sveta da poštanska ustanova želi da na jedan teren pošalje više od jednog poštara. 

Kao ulazne parametre ćemo imati neusmeren graf *G=(V, E)*, težine grana *w*, polazni čvor *s* i broj poštara *k*. Cilj je da se svaka grana obiđe bar jednom, ali tako da svaki poštar ima približno jednaku rutu. S obzirom da rešavamo NP-tešku varijantu problema, potrebno je minimizovati najdužu od k ruta.

### Algoritam grube sile

Pomoću funkcije *find_all_cycles(graph, start, end)* ćemo pronaći sve cikluse u zadatom grafu. Parametri koje prima funkcija su:

- *graph* - početni graf predstavljen kao rečnik
- *start* - početni čvor
- *end* - krajnji čvor

Izlaz funkcije su svi ciklusi koji postoje u grafu.

In [10]:
# Funkcija koja pronalazi svaki ciklus u grafu
def find_all_cycles(graph, start, end):
    fringe = [(start, [])]
    while fringe:
        state, path = fringe.pop()
        if path and state == end:
            yield path
            continue
        for next_state in graph[state]:
            if next_state in path:
                continue
            fringe.append((next_state, path+[next_state]))

Za predstavljanje grafa koristićemo rečnike *nodes* i *edges*:

- *nodes* - rečnik oblika čvor: lista_suseda
- *edges* - rečnik oblika čvor: cena_do_suseda

Oba rečnika su organizovana tako da i-ti član rečnika *nodes* odgovara i-tom članu rečnika *edges*. Tačnije, cena do suseda *n<sub>i</sub>* (iz rečnika *nodes*) je *e<sub>i</sub>* (iz rečnika *edges*).

In [ ]:
# Susedi svakog čvora u grafu
nodes = { 
    1: [2, 3, 7], 
    2: [1, 3, 4, 5], 
    3: [1, 2, 4, 5], 
    4: [2, 3, 6, 7, 8], 
    5: [2, 3, 6, 8],
    6: [4, 5, 8],
    7: [1, 4, 8],
    8: [4, 5, 6, 7]
}

# Cene iz čvorova ka čvorovima koji su navedeni iznad
edges = {
    1: [38, 1, 2],
    2: [38, 8, 10, 13],
    3: [1, 8, 26, 2],
    4: [10, 26, 8, 24, 1],
    5: [13, 2, 1, 7],
    6: [8, 1, 7],
    7: [2, 24, 27],
    8: [1, 7, 7, 27]
}

S obzirom da funkcija *find_all_cycles* vraća sve cikluse u grafu, pomoću funkcije *same_start_end* izolujemo samo one koji počinju i završavaju se čvorom 1. 
Izlaz iz funkcije je lista ruta koje počinju i završavaju se čvorom 1.

In [ ]:
# Funkcija koja vraća samo cikluse koji kreću i završavaju se u čvoru 1
def same_start_end(cycles):
    result_cycles = []
    for i in range(len(cycles)):
        if cycles[i][0] == 1 and cycles[i][-1] == 1:
            result_cycles.append(cycles[i])
    return result_cycles

Nakon toga, pomoću funkcije *calculate_paths* računamo ukupnu cenu svih tih ciklusa. Izlaz iz funkcije je lista parova (putanja, cena).

In [ ]:
# Funkcija koja računa cenu ture
def calculate_paths(cycles, edges):
    calculated_paths = []
    # Prolazimo kroz sve cikluse koji počinju i završavaju se sa 1
    for i in range(len(cycles)):
        path_cost = 0
        # Prolazimo kroz elemente jednog ciklusa
        for j in range(len(cycles[i])):
            if j != len(cycles[i]) - 1:
                current_node = cycles[i][j] # trenutni čvor
                index_of_cost = nodes[current_node].index(cycles[i][j+1]) # tražimo indeks suseda trenutnog čvora
                cost_to_add   = edges[cycles[i][j]][index_of_cost] # na osnovu indeksa, uzimamo cenu puta do tog suseda
                path_cost += cost_to_add # dodajemo tu cenu na ukupnu cenu ture
        
        calculated_paths.append((cycles[i], path_cost))
    
    
    return calculated_paths

Konačno, implementiramo funkciju *kcpp(calculated_paths, k_postmen)* koja prima listu parova (putanja, cena) i broj poštara koji treba da obiđu graf. 
Rezultat je lista k putanja i njihovih cena, kao i lista čvorova koje su poštari obišli (potrebno je da na ovom spisku budu svi čvorovi).

In [13]:
import random

depot_node_cycles = same_start_end(cycles)
k_postmen = 3
calculated_paths = calculate_paths(depot_node_cycles, edges)

def kcpp(calculated_paths, k_postmen):
    final_paths = []
    cities_visited = []
    
    for i in range(k_postmen):
        
        # Slučajnim izborom biramo jednu od putanja
        random_path_index = random.randrange(0, len(calculated_paths))
        chosen_path = calculated_paths[random_path_index]
        final_paths.append(chosen_path)
        
        # Za svaki čvor proveravamo da li je posećen i ukoliko nije, dodajemo ga u spisak posećenih
        for node in chosen_path[0]:
            if node not in cities_visited:
                cities_visited.append(node)
        
        # Ukoliko do poslednjeg poštara nismo obišli sve čvorove, potrebno je da on obiđe ostatak
        if i == k_postmen-1:
            cities_visited.sort()
            while cities_visited != list(nodes):
                last_path = final_paths[-1]
                final_paths.remove(last_path)
                random_path_index = random.randrange(0, len(calculated_paths))
                chosen_path = calculated_paths[random_path_index]
                final_paths.append(chosen_path)
                for node in chosen_path[0]:
                    if node not in cities_visited:
                        cities_visited.append(node)
            
            
    return final_paths, cities_visited

kcpp(calculated_paths, 2)

([([1, 7, 4, 8, 6, 5, 2, 3, 1], 57), ([1, 7, 8, 4, 2, 5, 3, 1], 56)],
 [1, 2, 3, 4, 5, 6, 7, 8])

Dalji cilj je optimizovati najdužu od putanja.